# Wissensaggregator Mittelalter und frühe Neuzeit

## Daten für Bischöfe einlesen

Übertrage Daten aus Access. Daten als CSV speichern.

[Vorspann](#Vorspann)  
[Litratur](#Literatur)  
[Item](#Item)  
[Person](#Person)  
[Externe ID](#Externe-ID)  
[Namensvarianten](#Namensvarianten)  
[Suchvarianten](#Suchvarianten)  

Quelle:   
*2022-05-26* Datenbank `gso_in_202207`  
*2022-10-19* Datenbank `gso_in_202210`

### Vorspann

In [1]:
wds_path="C:\\Users\\Georg\\Documents\\projekte\\WiagDataSetup.jl"

"C:\\Users\\Georg\\Documents\\projekte\\WiagDataSetup.jl"

In [2]:
cd(wds_path)

In [3]:
using Pkg

In [4]:
Pkg.activate(".")

  Activating project at `C:\Users\Georg\Documents\projekte\WiagDataSetup.jl`


In [5]:
using Revise

In [6]:
using WiagDataSetup

In [7]:
Wds = WiagDataSetup

WiagDataSetup

In [8]:
db = "wiag_in"

"wiag_in"

In [9]:
Wds.setDBWIAG(user = "georg", db = db)

Passwort für User georg: ········


MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag_in")

In [10]:
data_path = "C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb2\\data\\bischoefe-2022-10-19"

"C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb2\\data\\bischoefe-2022-10-19"

In [11]:
gs_db = "gso_in_202210"

"gso_in_202210"

In [12]:
item_type_id = 9 # Bischof GS       | Bischof des Alten Reiches aus der Personendatenbank der Germania Sacra |

9

In [13]:
ENV["COLUMNS"] = 120

120

In [14]:
using CSV, DataFrames, MySQL

In [15]:
includet("src/Util.jl")

In [16]:
includet("src/Bischof-GS.jl")

LoadError: LoadError: "unexpected \"end\""
in expression starting at C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Bischof-GS.jl:111

### Tabellen löschen
Tabelle `item` muss zuletzt gelöscht werden! (ausgenommen: `reference_volume`, `person` mit `item_type_id`)

In [17]:
clear_table!(Wds.dbwiag, "id_external", "item_id")

┌ Info: Zeilen 
│   n_row = 2377
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Util.jl:45


2377

In [18]:
clear_table!(Wds.dbwiag, "familyname_variant", "person_id")

┌ Info: Zeilen 
│   n_row = 285
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Util.jl:45


285

In [19]:
clear_table!(Wds.dbwiag, "givenname_variant", "person_id")

┌ Info: Zeilen 
│   n_row = 575
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Util.jl:45


575

### Literatur

In [35]:
table = "reference_volume"

"reference_volume"

In [36]:
count_table(Wds.dbwiag, table, "item_type_id")

┌ Info: SELECT COUNT(*) as n FROM reference_volume WHERE item_type_id = 9
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Util.jl:24


89

In [23]:
c = clear_table!(Wds.dbwiag, table, "item_type_id")

┌ Info: Zeilen 
│   n_row = 89
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Util.jl:45


89

In [24]:
source_table = "books"
insert_reference_volume!(table, source_table)

┌ Info: reading 
│   source_table = books
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Bischof-GS.jl:25
┌ Info: Zeilen, Spalten: 
│   size(df) = (89, 6)
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Bischof-GS.jl:34
┌ Info: Rows inserted: 89
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1216


89

### Item

In [46]:
table = "item"
count_table(Wds.dbwiag, table, "item_type_id")

┌ Info: SELECT COUNT(*) as n FROM item WHERE item_type_id = 9
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Util.jl:24


0

In [39]:
clear_table!(Wds.dbwiag, table, "item_type_id")

┌ Info: Zeilen 
│   n_row = 1368
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Util.jl:45


1368

In [40]:
dbwiag_name = db

"wiag_in"

In [41]:
df_gsn = gsn(dbwiag_name);

┌ Info: SELECT item.id as ep_item_id, e.value AS gsn
│     FROM wiag_in.item
│     JOIN wiag_in.id_external AS e ON item.id = e.item_id
│     WHERE item_type_id = 4
│     AND authority_id = 200
│     AND item.is_online
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Bischof-GS.jl:66


In [42]:
size(df_gsn)

(2143, 2)

In [27]:
names(df_gsn)

2-element Vector{String}:
 "ep_item_id"
 "gsn"

In [43]:
df_item = insert_item!(table, "gsn", "items", "persons", df_gsn, user_id = 40);

In [44]:
names(df_item)

5-element Vector{String}:
 "gsn_id"
 "item_id"
 "nummer"
 "status"
 "ep_item_id"

In [45]:
size(df_item)

(2143, 5)

fülle `df_p`

In [40]:
source_file = joinpath(data_path, "tbl_bischoefe_gatz.csv");
df_p = insert_item!(
    table,
    source_file, 
    online_status = "fertig", 
    id_public_key = "Pers-EPISCGatz",
    user_id = 40
);

┌ Info: Zeilen, Spalten: 
│   size(df_p) = (5171, 30)
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Bishop.jl:107
┌ Info: Rows inserted: 5171
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1216


### Person

In [41]:
table = "person"

"person"

In [42]:
count_table(Wds.dbwiag, table, "item_type_id")

5171

In [43]:
clear_table!(Wds.dbwiag, table, "item_type_id")

┌ Info: Zeilen: 
│   n_row = 5171
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Util.jl:43


5171

In [44]:
insert_person!(table, df_p)

┌ Info: unbekannte Ordensbezeichnungen
│   unique(unknown_order) = ["OSB (?)", "OFM ?", "OP/OPraem/Ocart", "Omel", "OESA/OSB", "Ocist", "OPraem/OP", "OFM?", "OFM?, OCist", "OCist?", "OSB?", "OP?", "1780", "OFM oder OESA", "Ocarm", "OMel/OCist", "OFM (?)", "OP ?", "OFM (OESA?)", "OP/OFM"]
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Bishop.jl:138
┌ Info: Rows inserted: 5171
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1216


5171

### Externe ID

In [47]:
table = "id_external"

"id_external"

In [48]:
count_table(Wds.dbwiag, table, "item_id")

0

In [50]:
clear_table!(Wds.dbwiag, table, "item_id")

┌ Info: Zeilen 
│   n_row = 0
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Util.jl:43


0

GSN

In [55]:
gso_db = "gso_in_202210";
df_ep_gsn, df_info = join_gsn(df_p, gso_db);

In [56]:
df_info

Row,id_in_source,Vorname,Familienname,GSN_ID,nummer_neu
,String?,String,String?,String?,String
1,20711,Johannes,Schneider,026-04306-001,066-04915-001


In [58]:
table = "id_external"

"id_external"

In [57]:
insert_gsn!(table, df_ep_gsn::AbstractDataFrame)

┌ Info: Rows inserted: 2143
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1216


2143

VIAF, Wikidata, GND

In [60]:
insert_id_external!(table, df_p)

┌ Info: Normdatum 
│   auth.second = GND_ID
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Bishop.jl:224
┌ Info: Rows inserted: 2866
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1216
┌ Info: Normdatum 
│   auth.second = Wikidata_ID
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Bishop.jl:224
┌ Info: Rows inserted: 5171
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1216
┌ Info: Normdatum 
│   auth.second = VIAF_ID
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Bishop.jl:224
┌ Info: Rows inserted: 2972
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1216


Wikipedia

In [79]:
df_ep = insert_wikipedia_url!(table, df_p);

┌ Info: Rows inserted: 2940
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1216


### Namensvarianten

In [86]:
table = "familyname_variant"
col = :Familienname_Variante

:Familienname_Variante

In [90]:
count_table(Wds.dbwiag, table, "person_id")

698

In [91]:
clear_table!(Wds.dbwiag, table, "person_id")

┌ Info: Zeilen 
│   n_row = 698
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Util.jl:43


698

In [92]:
insert_name_variant!(table, df_p, col)

┌ Info: Rows inserted: 698
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1216


698

In [102]:
table = "givenname_variant"
col = :Vorname_Varianten

:Vorname_Varianten

In [99]:
count_table(Wds.dbwiag, table, "person_id")

0

In [100]:
clear_table!(Wds.dbwiag, table, "person_id")

┌ Info: Zeilen 
│   n_row = 0
└ @ Main C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\Util.jl:43


0

In [103]:
insert_name_variant!(table, df_p, col)

┌ Info: Rows inserted: 330
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1216


330

### Suchvarianten

In [104]:
table = "name_lookup"

"name_lookup"

In [105]:
count_table(Wds.dbwiag, table, "person_id")

0

In [106]:
insert_name_lookup!(table, df_p)

┌ Info: Rows inserted: 7314
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1216


7314